In [ ]:
include("problem.jl")
include("../fileIO.jl")

using Plots
using Random
Random.seed!(12);

# 2D Free-Flyer Robot System

## Problem Statement

In [ ]:
# Continuous time system, state x = [x, y, xdot, ydot] control u = [ux, uy]
n = 2;
Ac = [zeros(n,n) I;
      zeros(n,2*n)]
Bc = [zeros(n,n) I]'

dh = 0.75
A, B = c2d(Ac, Bc, dh)

N =6 # horizon
m = size(B, 2)
Q = diagm([2.;2;1;1])
R = 0.1*Matrix{Float64}(I,n,n)
mass_ff_min = 15.36
mass_ff_max = 18.08
mass_ff = 0.5*(mass_ff_min+mass_ff_max)
thrust_max = 2*1.  # max thrust [N] from two thrusters 
umin = -thrust_max/mass_ff
umax = thrust_max/mass_ff
velmin = -0.2
velmax = 0.2
posmin = [0;0]
ft2m = 0.3048
posmax = ft2m*[12.,9.]
max_box_size = 0.75
min_box_size = 0.25
box_buffer = 0.025
border_size = 0.05

## Compute Upper Bound for the problem

In [ ]:
UB, solved = upperBoundObstacleAvoidanceProb()

## Collect Dataset
Vary obstacles position and size and initial position of the agent (both within certain bounds).

In [ ]:
function findIC(obstacles)
    IC_found = false
    global x0
    while !IC_found
        global x0 = [posmin .+ (posmax-posmin).*rand(2); velmin .+ (velmax-velmin).*rand(2)]
        if !any([x0[1] >= obstacle[1][1] && x0[1] <= obstacle[1][2] && 
                    x0[2] >= obstacle[2][1] && x0[2] <= obstacle[2][2] for obstacle in obstacles])
            IC_found = true
        end
    end
    return x0
end

function findObs(x0,Nobstacles; max_iter=50)
    obs = []
    itr = 0
    while length(obs) < Nobstacles && itr < max_iter
        xmin = (posmax[1] - border_size - max_box_size)*rand() + border_size
        xmin = max(xmin, x0[1])
        xmax = xmin + min_box_size  + (max_box_size - min_box_size)*rand()
        ymin = (posmax[2] - border_size - max_box_size)*rand() + border_size
        ymin = max(ymin, x0[2])
        ymax = ymin + min_box_size  + (max_box_size - min_box_size)*rand()
        obstacle = [[xmin - box_buffer, xmax + box_buffer], 
                    [ymin - box_buffer, ymax + box_buffer]]
        if is_free_state(x0, [obstacle])
            push!(obs, obstacle)
        end
        itr+=1
    end
    if length(obs) != Nobstacles
        return []
    end
    obs
end

function is_free_state(x0,obstacles)
    if any([x0[1] >= obstacle[1][1] && x0[1] <= obstacle[1][2] && 
                    x0[2] >= obstacle[2][1] && x0[2] <= obstacle[2][2] for obstacle in obstacles])
        return false
    end
    return true
end

In [ ]:
# # Random.seed!(133)

# rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

# Nobstacles = 8
# x0 = [posmin .+ (posmax-posmin).*rand(2); velmin .+ (velmax-velmin).*rand(2)]
# obs = findObs(x0, Nobstacles)

# xg = [0.9*posmax; zeros(n)]
# x, u, y, cost, solve_time, node_count, solver_success = solveObstacleAvoidanceProb(A, B, Q, R, N, 
#                                                         umin, umax, velmin, velmax, 
#                                                         posmin, posmax, obs, x0, xg);

# if length(obs) != Nobstacles
#     @warn "Soln not found!"
# end

# Plots.quiver([x0[1]], [x0[2]], gradient=([x0[3]], [x0[4]]))
# if solver_success
#     @warn node_count
#     @warn solve_time
#     Plots.quiver(x[1,:], x[2,:], gradient=(x[3,:],x[4,:]))
# else
#     @warn "Solver failed"
# end

# for oo in obs[1:end-1]
#     Plots.plot!(rectangle(oo[1][2]-oo[1][1],oo[2][2]-oo[2][1],oo[1][1], oo[2][1]), opacity=0.5, legend=false)
# end
# oo = obs[end]
# Plots.plot!(rectangle(oo[1][2]-oo[1][1],oo[2][2]-oo[2][1],oo[1][1], oo[2][1]), opacity=0.5, legend=false, xaxis=("", (0,posmax[1])),yaxis=("", (0,posmax[2])))

In [ ]:
Nsets = 20
Ndata = 5000
Nobstacles = 4

obs_new_ct = 10
toggle_ct = 50

x0 = [ft2m*ones(2); zeros(2)]
xg = [0.9*posmax; zeros(n)]
obstacles = []

for jj = 1:Nsets
    # Problem
    probdata = Dict("A"=>A, "B"=>B, "N"=>N, 
            "umin"=>umin, "umax"=>umax, 
            "velmin"=>velmin, "velmax"=>velmax, 
            "posmin"=>posmin, "posmax"=>posmax, 
            "Q"=>Q, "R"=>R,
            "max_box_size"=>max_box_size, "min_box_size"=>min_box_size,
            "border_size"=>border_size, "UB"=>UB, "box_buffer"=>box_buffer);

    X0 = []
    Xg = []
    O = []
    X = []
    Y = []
    U = []
    J = Float64[]
    solution_times = Float64[]
    node_counts = Int[]

    ii = 0
    ii_obs = 0

    toggle_obstacles = true
    ii_toggle = 0 

    while ii < Ndata
        if toggle_obstacles
            global x0 = [posmin .+ (posmax-posmin).*rand(2); velmin .+ (velmax-velmin).*rand(2)]
            global obstacles = findObs(x0, Nobstacles)
            if length(obstacles) == 0
                continue
            end

            if mod(ii_toggle,obs_new_ct) == 0
                toggle_obstacles = false
                ii_obs = 0
                ii_toggle = 0
            end
        else
            # Generate obstacles
            if mod(ii_obs,obs_new_ct) == 0
                global obstacles = []
                for _ in 1:Nobstacles
                    xmin = (posmax[1] - border_size - max_box_size)*rand() + border_size
                    xmax = xmin + min_box_size  + (max_box_size - min_box_size)*rand()
                    ymin = (posmax[2] - border_size - max_box_size)*rand() + border_size
                    ymax = ymin + min_box_size  + (max_box_size - min_box_size)*rand()
                    obstacle = [[xmin - box_buffer, xmax + box_buffer], 
                                [ymin - box_buffer, ymax + box_buffer]]
                    push!(obstacles, obstacle)
                end
            end

            # Generate initial condition (that is outside of obstacles)
            global x0 = findIC(obstacles)

            if mod(ii_toggle,toggle_ct) == 0
                toggle_obstacles = true
                ii_obs = 0
                ii_toggle = 0
            end
        end

        # Solve Problem
        x, u, y, cost, solve_time, node_count, solver_success = solveObstacleAvoidanceProb(A, B, Q, R, N, 
                                                        umin, umax, velmin, velmax, 
                                                        posmin, posmax, obstacles, x0, xg);

        # Save all the data
        if solver_success
            push!(X0, x0)
            push!(Xg, xg)
            push!(O, obstacles)
            push!(X, x)
            push!(Y, y)
            push!(U, u)
            push!(J, cost)
            push!(solution_times, solve_time)
            push!(node_counts, node_count)

            ii += 1
            ii_obs += 1
            ii_toggle += 1
            print(string(ii,"/",Ndata,"\r"))
        end
    end

    base = "data/testdata"
    filename = string(base, string(jj), ".jld")

    training_data = Dict("X0"=>X0, "Xg"=>Xg, "O"=>O, "X"=>X, "Y"=>Y,
        "U"=>U, "J"=>J, 
        "solve_time"=>solution_times, "node_count"=>node_counts,
        "prob"=>probdata)
    writeTrainingData(filename, training_data)
    println(string("\n",jj))
end